In [1]:
import time
import numpy as np
import pandas as pd
import os

# from hydesign.assembly.hpp_assembly import hpp_model
# from hydesign.examples import examples_filepath

In [ ]:
sites = pd.read_csv('overview.csv', index_col=0)
example = 0

ex_site = sites.iloc[int(example),:]

print('Selected example site:')
print('---------------------------------------------------')
print(ex_site.T)

longitude = ex_site['longitude']
latitude = ex_site['latitude']
altitude = ex_site['altitude']
input_ts_fn = os.path.join(os.getcwd(), ex_site['input_ts_fn'])
sim_pars_fn = os.path.join(os.getcwd(), ex_site['sim_pars_fn'])

hpp = hpp_model(
        latitude,
        longitude,
        altitude,
        num_batteries = 10,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
        save_battry_rf_filename='battery_rf.csv',
)

inputs = dict(
clearance =	60,
sp =	287.6391934,
p_rated =	10,
Nwt	=	25,
wind_MW_per_km2 =	4.782274215,
solar_MW =	223.2191223,
surface_tilt =	50,
surface_azimuth = 210,
DC_AC_ratio	=1.295598428,
b_P =75.27402422,
b_E_h=	4.451612903,
cost_of_battery_P_fluct_in_peak_price_ratio	=0.580645161,)

In [ ]:
start = time.time()
outs = hpp.evaluate(**inputs)
hpp.print_design([v for k,v in inputs.items()], outs)
end = time.time()
print('exec. time [min]:', (end - start)/60 )
print(hpp.prob['NPV_over_CAPEX'])

In [2]:
from hopp.simulation import HoppInterface

/Users/kbrunik/github/tool-benchmarking/Europe/log/hybrid_systems_2024-05-02T10.58.02.357875.log


/Users/kbrunik/anaconda3/envs/gh/lib/python3.8/site-packages/fastkml/config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")


In [3]:
hi = HoppInterface(os.path.join(os.getcwd(), "hopp_input/08-wind-solar-france.yaml"))

FLORIS is the system model...


In [4]:
hi.simulate(25)

Simulating wind farm output in FLORIS...
Wind annual energy:  663116990.2109435


In [8]:
hybrid_plant = hi.system

annual_energies = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
battery_installed_cost = hybrid_plant.battery.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

print("Wind Installed Cost [M$]: {}".format(wind_installed_cost/1e6))
print("Solar Installed Cost [M$]: {}".format(solar_installed_cost/1e6))
print("Battery Installed Cost [M$]: {}".format(battery_installed_cost/1e6))
print("Hybrid Installed Cost [M$]: {}\n".format(hybrid_installed_cost/1e6))

print("Wind NPV [M$]: {}".format(hybrid_plant.net_present_values.wind/1e6))
print("Solar NPV [M$]: {}".format(hybrid_plant.net_present_values.pv/1e6))
print("Hybrid NPV [M$]: {}\n".format(hybrid_plant.net_present_values.hybrid/1e6))

print("Annual Energies [kW]",annual_energies)
print("Capacity Factors",cf)
print("NPV [$]", npvs)

print("LCOE [cents/kWh]", hybrid_plant.lcoe_real,"\n")

print("Hybrid LCOE [$/MWh]", hybrid_plant.lcoe_real['hybrid']*10)

Wind Installed Cost: 241.227
Solar Installed Cost: 54.989034800000006
Battery Installed Cost: 17.5693665
Hybrid Installed Cost: 313.7854013

Wind NPV: 43.13380019290942
Solar NPV: 2.545612169117138
Hybrid NPV: 39.987899363889575

Annual Energies [kW] {"pv": 277789162.6264313, "wind": 663116990.2109435, "battery": -2974975.4070675476, "hybrid": 936708992.6552676}
Capacity Factors {"pv": 18.486286596963403, "wind": 30.279314621504266, "battery": 0, "hybrid": 21.563237776446694}
NPV [$] {"pv": 2545612.169117138, "wind": 43133800.19290942, "battery": -1907778.2418845985, "hybrid": 39987899.363889575}
LCOE [cents/kWh] {"pv": 2.4773060420501594, "wind": 2.973920041501466, "battery": 4.9791566373582095, "hybrid": 3.1015048262713227} 

Hybrid LCOE [$/MWh] 31.01504826271323


In [7]:
import json

with open("singleowner.json", 'w') as f:
    dat = hybrid_plant.grid._financial_model.export()
    d = dict()
    for k, v in dat.items():
        d.update(v)
    json.dump(d, f)